In [3]:
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_parquet('/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-01.parquet')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [8]:
df.duration = df['duration'].apply(lambda dt: dt.total_seconds())

In [9]:
df.duration = (df.duration / 60)

In [10]:
df['duration'].describe(percentiles = [0.95,0.99,0.05])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
5%       3.300000e+00
50%      1.151667e+01
95%      3.646667e+01
99%      5.725000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [11]:
df_reduced = df[(df['duration'] >= 1.0) & (df['duration'] <= 60.0)]

In [12]:
df_reduced.shape[0] / df.shape[0]

0.9812202822125979

In [13]:
df_reduced.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [14]:
categorical = ['PULocationID', 'DOLocationID']
data = df_reduced[categorical].astype(str).to_dict(orient='records')
vec = DictVectorizer()
train_data = vec.fit_transform(data[:20000])

In [15]:
train_data.shape

(20000, 369)

In [16]:
labels = df_reduced['duration'][:20000]

In [17]:
lr = LinearRegression()
lr.fit(train_data, labels)

LinearRegression()

In [18]:
preds = lr.predict(train_data)
preds

array([11.78486442, 10.12042844, 16.5317472 , ..., 12.30941964,
       11.59861344, 17.31574591])

In [19]:
mean_squared_error(labels, preds, squared=False)

7.278400150799101

In [20]:
val_df = pd.read_parquet('/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-02.parquet')

In [22]:
val_df['duration'] = (val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']).apply(lambda x: x.total_seconds() / 60) 


In [23]:
val_df = val_df[(val_df['duration'] >= 1) & (val_df['duration'] <= 60)][:10000]

In [24]:
val_data = val_df[categorical].astype(str).to_dict(orient='records')
val_data = vec.transform(val_data)

In [25]:
val_preds = lr.predict(val_data)

In [26]:
val_labels = val_df['duration']

In [27]:
mean_squared_error(val_labels, val_preds, squared=False)

8.424044306419704